In [ ]:
#Getting access to MP

from mpcontribs.client import Client
import os

os.environ['MPCONTRIBS_API_KEY']='BEuiG4rxRKQR4M4q4S3tyHW7Y516a6Iz'
os.environ['SSL_CERT_FILE']='/Users/minhyukkang/opt/anaconda3/envs/TKenv/lib/python3.9/site-packages/certifi/cacert.pem'
client=Client(project='qmof')

In [ ]:
#Creating HSE data

import pandas as pd
P4Mt = ['K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn']
not_exist = []
HSE_list = []
for i in P4Mt:
    query_HSE = {'project':'qmof','formula__contains':i, '_fields':['id','formula','data.EgHSE06.value'], '_limit':500}
    contribs_HSE = client.contributions.get_entries(**query_HSE).result()

    contribs_HSE = contribs_HSE['data'].copy()
    pd_HSE = pd.DataFrame(contribs_HSE)

    n = 0
    try:
        for j in pd_HSE['formula']:
            HSE_value = float(str(pd_HSE['data'][n]).replace('{\'EgHSE06\': {\'value\': ','').replace('}}',''))
            if HSE_value < 1.0:
                HSE_list.append([pd_HSE['id'][n], j, float(str(pd_HSE['data'][n]).replace('{\'EgHSE06\': {\'value\': ','').replace('}}',''))])
            n = n+1
    except:
        HSE_list.append([j, 0])
exec(f'pd_HSE_id = pd.DataFrame(HSE_list, columns = [\'id\',\'formula\', \'HSE\'])')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
exec(f'print(pd_HSE_id)')

In [ ]:
# Making a list/table showing whether two molecules are geometrically similar or not

from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.core.structure import Structure
tb = []
n = 0
for i in pd_HSE_id['id']:
    sp1 = client.get_structure(client.get_contribution(i)['structures'][0]['id'])
    tol = [pd_HSE_id['formula'][n]]
    for j in pd_HSE_id['id']:
        sp2 = client.get_structure(client.get_contribution(j)['structures'][0]['id'])
        sm = StructureMatcher(ltol= 0.3, stol= 3, angle_tol=360, primitive_cell=False, scale=True, attempt_supercell=True, allow_subset=True, comparator=None, ignored_species=None)
#        while sm.fit_anonymous(sp1, sp2) == False and m < 10:
#            sm = StructureMatcher(ltol= 0.3, stol= m , angle_tol=360, primitive_cell=False, scale=True, attempt_supercell=True, allow_subset=True, comparator=None, ignored_species=None)
#            m = m + 1
        if sm.fit_anonymous(sp1, sp2) == True:
            tol.append(1)
        else:
            tol.append(0)
    n = n+1
    print(tol)
    tb.append(tol)
print(tb)        
        
        
#sp1 = Structure.from_file('/Users/minhyukkang/Metal Structures/Co_Structures/geometry_CoC40H52N2O4.cif')
#sp2 = Structure.from_file('/Users/minhyukkang/Metal Structures/Co_Structures/geometry_CoC40H50N2O4.cif')

In [ ]:
#Converting the list into dataframe table

ttl = ['\\']
for i in pd_HSE_id['formula']:
    ttl.append(i)
pd_tb = pd.DataFrame(tb, columns = ttl)    
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd_tb

#pd_tb.to_csv('/Users/minhyukkang/Downloads/Low_Bandgap_Similarity.csv')

In [ ]:
#Dimension alaysis test (Comparing two different functions)

import pymatgen.analysis.dimensionality as dim
dim_list = []
n = 0
for i in pd_HSE_id['id']:
    sp = client.get_structure(client.get_contribution(i)['structures'][0]['id'])
    dim_list.append([pd_HSE_id['formula'][n], dim.get_dimensionality_gorai(sp), dim.get_dimensionality_cheon(sp)])
    n = n+1
pd.DataFrame(dim_list, columns = ['Formula', 'Dimensionality Test Gorai', 'Dimensionality Test Cheon'])

In [ ]:
#Creating HSE data
import pandas as pd
P4Mt = ['Cu'] #['K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn']
not_exist = []

query_PBE = {'project':'qmof','formula__contains':'Cu', '_fields':['id','formula','data.EgPBE.value'], '_limit':500}
contribs_PBE = client.contributions.get_entries(**query_PBE).result()

contribs_PBE = contribs_PBE['data'].copy()
#print(contribs_PBE)
pd_PBE = pd.DataFrame(contribs_PBE)
#print(pd_PBE)
PBE_list = []
n = 0
for i in range(300, 500):
    j = pd_PBE['formula'][i]
    try:
        sp = client.get_structure(client.get_contribution(pd_PBE['id'][n])['structures'][0]['id'])
        dm = dim.get_dimensionality_gorai(sp)
        PBE_list.append([j, float(str(pd_PBE['data'][n]).replace('{\'EgPBE\': {\'value\': ','').replace('}}','')), dm])
        n = n+1
    except:
        PBE_list.append([j,0,0])
        n = n+1
pd_PBE_Cu_1 = pd.DataFrame(PBE_list,columns = ['formula', 'PBE', 'Dim']).sort_values(by = 'PBE')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(pd_PBE_Cu_1)

In [ ]:
#Creating a list and box-and-whisker plot of Bandgap(PBE) and Dimension for MOFs

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymatgen.analysis.dimensionality as dim

P4Mt = ['Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Zn']
for i in P4Mt:
    query_PBE = {'project':'qmof','formula__contains':i, '_fields':['id','formula','data.EgPBE.value'], '_limit':500}
    contribs_PBE = client.contributions.get_entries(**query_PBE).result()

    contribs_PBE = contribs_PBE['data'].copy()
    #print(contribs_PBE)
    pd_PBE = pd.DataFrame(contribs_PBE)
    #print(pd_PBE)
    PBE_list = []
    n = 0
    for j in pd_PBE['formula']:
        try:
            sp = client.get_structure(client.get_contribution(pd_PBE['id'][n])['structures'][0]['id'])
            dm = dim.get_dimensionality_gorai(sp)
            PBE_list.append([j, float(str(pd_PBE['data'][n]).replace('{\'EgPBE\': {\'value\': ','').replace('}}','')), dm])
            n = n+1
        except:
            PBE_list.append([j,0,0])
            n = n+1
    exec(f'pd_PBE_{i} = pd.DataFrame(PBE_list,columns = [\'formula\', \'PBE\', \'Dim\']).sort_values(by = \'PBE\')')
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    exec(f'print(pd_PBE_{i})')
    exec(f'sns.boxplot(x = \'Dim\', y = \'PBE\', data = pd_PBE_{i})')
    #plt.scatter(pd_PBE_Cu['Dim'], pd_PBE_Cu['PBE'])
    plt.xticks([2,3])
    plt.xlim([1.5,3.5])
    plt.xlabel('Dimensionality')
    plt.ylabel('Bandgap_PBE (eV)')
    exec(f'plt.title(\'{i} Bandgap\')')
    exec(f'plt.show()')